# Algoritmia
## Práctica 2 - Divide y Vencerás
### Curso 2023 - 2024
---
 

#### Autores:
* Cristian Fernández Martínez
* Alicia García Pérez

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.8/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [688]:
#testeable
# imports

In [689]:
#testeable
# Versión ligeramente modificada de la clase Video de la Práctica 1

# Se ha incluido una propiedad 'fee', correspondiente a los costes 
# semanales de licenciamiento del contenido
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size, fee):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        self.fee = fee
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
    
        return hash((self.name, self.size, self.fee))
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """    
        return f'Nombre del video: {self.name}, tamaño: ({self.size} MB), coste licenciamiento: {self.fee}'
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f'Nombre del video: {self.name}, tamaño: ({self.size} MB), coste licenciamiento: {self.fee}'
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """        
        
        self.users[country] = users
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadore s para el país `country`
        """        
    
        return self.users.get(country, 0)
    
    def __lt__(self,other):
        return self.size < other.size
    
    def __le__ (self,other):
        return self.size <= other.size



In [690]:
#testeable
def key_sort(seq, key_function= lambda x: x, reverse=False):
    """Ordena ascendentemente una secuencia utilizando una función clave.

    Parameters
    ----------
    seq : sequence
        Secuencia a ordenar, por ejemplo una lista
    key_function : int
        Función que devuelve para los elementos de la secuencia un valor
        clave para la ordenación
    reverse: boolean
        Si es True Invierte el orden de la ordenación.
    
    Notes
    -----
    No tiene retorno. Ordena en el sitio la secuencia de entrada.
    """      

    def quicksort(seq,izq,der):
        if izq < der:
            # Obtenemos el pivote  y realizamos quicksort en las mitades
            ultimo_izq,primero_der = particion(seq,izq,der)
            quicksort(seq,izq,ultimo_izq)
            quicksort(seq,primero_der,der)

    def particion(seq, izq, der):
        # Tomamos el elemento medio como pivote e i como el indice del menor elemento
        medio = (izq + der) // 2
        pivote = key_function(seq[medio])
        seq[medio], seq[der] = seq[der], seq[medio]
        i = izq - 1

        for j in range(izq, der):
            # Si el elemento actual es menor o igual al pivote
            if key_function(seq[j]) <= pivote:
                # Incrementamos el indice del menor elemento
                i = i + 1
                # Intercambiamos los elementos
                seq[i], seq[j] = seq[j], seq[i]

        # Intercambiamos el pivote con el elemento en el indice del menor elemento
        seq[i + 1], seq[der] = seq[der], seq[i + 1]
        return i, i + 2
    
    # Llamamos a la funcion de ordenacion
    quicksort(seq,0,len(seq)-1)

    if reverse==True:
        seq.reverse()

In [691]:
#testeable
def max_emission_profit(video, forecast, ingresos_por_usuario, method="bruteforce"):
    """Calcula el máximo beneficio obtenible emitiendo un vídeo según 
       la previsión de espectadores disponible.

    Parameters
    ----------
    video : Video
        Vídeo a emitir.
    forecast : iterable
        Previsión de espectadores.
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    method: String
        Algoritmo a emplear 'bruteforce' o 'divideandconquer'.
    
    Returns
    -------
    float
        Máximo beneficio obtenible.
        
    Notes
    -----
    El máximo beneficio se calcula considerando la mejor combinación 
    posible de momento de inicio y finalización dentro del periodo de 
    forecast, el coste de licenciamiento del vídeo (fee) y los beneficios 
    obtenidos por cada espectador. Si el vídeo no puede emitirse de forma
    rentable, el método devuelve 0.
    'video' debe disponer de una propiedad 'fee' que establece los costes
    de licenciamiento en periodos equivalentes a las previsiones de 'forecast'.
    """      
    if method == "bruteforce":
        return max_emission_profit_bruteforce(video, forecast, ingresos_por_usuario)
    elif method == "divideandconquer":
        return max_emission_profit_divideandconquer(video, forecast, ingresos_por_usuario, 0, len(forecast)-1)

#Parte de fuerza bruta
def max_emission_profit_bruteforce(video, forecast, ingresos_por_usuario):
    max_beneficio = 0
    n = len(forecast)

    for inicio in range(n):
        for fin in range(inicio, n):
            # Calculamos el beneficio para cada semana en el rango
            beneficio = sum(forecast[i] * ingresos_por_usuario - video.fee for i in range(inicio, fin + 1))
            max_beneficio = max(max_beneficio, beneficio)

    return max(0, max_beneficio)

#Parte de divide y venceras
def max_emission_profit_divideandconquer(video, forecast, ingresos_por_usuario, bajo, alto):
    # Comprobar que 'bajo' y 'alto' están dentro del rango de la lista 'forecast'
    if bajo < 0 or alto >= len(forecast):
        return 0
    
    if bajo > alto:
        return 0

    if bajo == alto:
        return max(0, forecast[bajo] * ingresos_por_usuario - video.fee)
    
    medio = bajo + (alto - bajo) // 2

    maximo_izq = max_emission_profit_divideandconquer(video, forecast, ingresos_por_usuario, bajo, medio)
    maximo_der = max_emission_profit_divideandconquer(video, forecast, ingresos_por_usuario, medio+1, alto)
    maximo_cruzado = beneficio_cruzado(video, forecast, ingresos_por_usuario, bajo, medio, alto)

    return max(0, maximo_izq, maximo_der, maximo_cruzado)

def beneficio_cruzado(video, forecast, ingresos_por_usuario, bajo, medio, alto):
    suma = 0
    maximo = 0

    # Calculamos el beneficio maximo de la parte izquierda
    for i in range(medio, bajo-1, -1):
        suma += forecast[i] * ingresos_por_usuario - video.fee
        maximo = max(maximo, suma)

    suma = maximo

    # Calculamos el beneficio maximo de la parte derecha
    for i in range(medio+1, alto+1):
        suma += forecast[i] * ingresos_por_usuario - video.fee
        maximo = max(maximo, suma)

    return max(0, maximo)

In [692]:
#testeable
def optimize_content_grid(videos, forecasts, ingresos_por_usuario, k=None):
    """Obtiene los vídeos rentables para su emisión según la previsión
    de espectadores y los ingresos por usuario.

    Parameters
    ----------
    videos : list of Video
        Vídeos a analizar.
    forecasts : list of list of int
        Secuencia de previsiones para los vídeos
    ingresos_por_usuario: float
        Ingresos brutos que proporciona cada usuario.
    k: int
        Obtiene los k elementos más rentables.
    
    Returns
    -------
    list
        Lista con los k elementos más rentables.
        
    Notes
    -----
    La rentabilidad de los vídeos se considera tomando el periodo
    óptimo de emisión para cada uno. El método sólo devuelve resultados
    con rentabilidad mayor que 0.
    """      
    beneficios = []
    for video in videos:
        for forecast in forecasts:
            beneficios.append(max_emission_profit(video, forecast, ingresos_por_usuario))
        video.beneficio = max(beneficios)
        beneficios.clear()

    key_sort(videos,key_function=lambda x: x.beneficio, reverse=True)

    #Quitar para la entrega
    for video in videos:
        print(video.name, video.beneficio)

    # Seleccionar las primeras `k` tuplas de la lista ordenada
    if k is not None:
        videos = videos[:k]

    return videos

### Caso de ejemplo

In [693]:
import unittest,random

# Vídeos de ejemplo para tests
videos_test = [Video("Vikings: Valhalla", 150, 1500), #Título, tamaño, fee
               Video("New Amsterdam", 56, 980),
               Video("Juego de Tronos",152, 2500),
               Video("La casa de papel", 76, 800),
               Video("Breaking Bad", 160, 1230),
               Video("Pasión de gavilanes", 125, 350),
               Video("The Sinner", 89, 1900),
               Video("El Cid", 35, 410),
               Video("Raised by wolves",80, 1900),
               Video("Euphoria",90, 2000),
               Video("Peacemaker",40, 1000)]

# Audiencias previstas para los anteriores
forecasts_test = [[ 500, 1000, 2000,  150, 2000, 1500,  200, 1200,  700],
                  [1000,  800,  500,  250, 4000,  500,  100,  200,  300],
                  [1200, 1000,  900,  900, 1220, 1100,  900, 1000, 1000],
                  [ 900,  700,  500,  200, 1600,  200,  300,  250,  100],
                  [ 600,  900, 1240,  250, 2100, 1000,  350, 1000,  800],
                  [ 190, 1060,  500,  200,  250,  700,  200,  400,  400],
                  [ 150, 1100, 2500,  600,  800, 3000,  200, 1000,  900],
                  [ 160,  100,  200,  200,  150,  200,   50,  200,  190],
                  [ 900,  850,  960,  950, 1100,  100,  350,  900,  800],
                  [ 600,  900, 1200,  300, 1900, 1000,  650,  850,  800],
                  [ 510,  500,  500,  500,  500,  500,  500,  500,  510]]

class TestBasico(unittest.TestCase):

    def test_basic_sort(self):
        v = list(range(100))
        random.seed = 1234
        random.shuffle(v)
        key_sort(v)
        self.assertEqual(v, list(range(100))) 

    def test_max_profit(self):   
        for m in ["bruteforce", "divideandconquer"]:
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [1]*10, 1, method=m), 0)
            self.assertEqual(max_emission_profit(Video("test", 99, 1), [2]*10, 1, method=m), 10)

        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [696]:
print(optimize_content_grid(videos_test, forecasts_test, 3,0))



Pasión de gavilanes 27600
El Cid 27060
La casa de papel 23900
New Amsterdam 22460
Peacemaker 22300
Breaking Bad 20460
Vikings: Valhalla 18300
The Sinner 15100
Raised by wolves 15100
Euphoria 14300
Juego de Tronos 11500
[]


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de la librería `nbformat`

```bash
$ pip install nbformat
```

###### Explicación de los tests
* `test_ej1_basic_sort`: Comprueba que se ordenen correctamente listas de números.
* `test_ej1_key_sort`: Comprueba que se ordenan correctamente listas según su función clave.
* `test_ej1_video_sort`: Comprueba que se pueden ordenar vídeos.
* `test_ej2_max_profit`: Comprueba que se obtiene el beneficio máximo tanto por fuerza bruta como por divide y vencerás para listas de números simples.
* `test_ej2_max_profit_dataset`: Comprueba que se obtiene el beneficio máximo tanto por fuerza bruta como por divide y vencerás para listas más complejas.
* `test_ej2_optimize_content_grid`: Comprueba el buen desempeño de la función `optimize_content_grid`.

---

### **Informe**
Contesta a las siguientes preguntas.

#### **Complejidad**

1. Método `key_sort`
    * **Complejidad temporal**: _contesta aquí_
2. Método `max_emission_profit`
    * **Complejidad temporal**: _contesta aquí_
2. Método `optimize_content_grid`
    * **Complejidad temporal**: _contesta aquí_


#### **Ranking de contenidos.**

* ¿Cómo afecta la función clave, que proporciona el criterio de ordenación, a la complejidad temporal del algoritmo?

_Contesta aquí justificando la respuesta_

#### **Optimización de la compra de contenidos.**

* ¿Has conseguido mejorar la complejidad temporal mediante la solución Divide y Vencerás? 
* ¿Por qué ha mejorado? ¿Cuánto ha mejorado? <br>
<sub>Pon algunos ejemplos numéricos especulativos que relacionen el tamaño del problema con el tiempo de ejecución en ambas versiones. Contrástalos de forma empírica empleando para ello tu implementación.</sub>

_Contesta aquí justificando la respuesta_